In [1]:
import json
import re
import warnings
from functools import lru_cache
from pathlib import Path

import aquarel
import ir_datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ir_datasets import docs_parent_id

import yaml
from scipy.stats import pearsonr, ttest_ind, ttest_rel
from tqdm import tqdm

from lightning_ir.lightning_utils.validation_utils import evaluate_run
from lightning_ir.data.dataset import DASHED_DATASET_MAP

In [2]:
theme = aquarel.Theme.from_file(Path.home() / "aquarel-theme.json")
theme.apply()
markers = ["o", "X", "s", "v", "P", "*", "D"]
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [3]:
BASELINE_DIR = Path.cwd().parent / "data/baseline-runs"
LOG_DIR = Path.cwd().parent / "experiments/wandb"

In [29]:
def parse_config(config_path):
    config = yaml.safe_load(config_path.read_text())

    flat_config = {}

    def flatten_config(config, prefix=""):
        for k, v in config.items():
            key = prefix + "." + k if prefix else k
            if isinstance(v, dict):
                if len(v) == 2 and "init_args" in v:
                    flat_config[key + f".class_path"] = v["class_path"]
                    flatten_config(v["init_args"], key)
                else:
                    flatten_config(v, key)
            if isinstance(v, (list, tuple)):
                for i, _v in enumerate(v):
                    if isinstance(_v, dict):
                        flatten_config(_v, f"{key}.{i}")
                    else:
                        flat_config[f"{key}.{i}"] = _v
            else:
                flat_config[key] = v
        return config

    flatten_config(config)

    config = json.loads(
        (config_path.parent / "huggingface_checkpoint" / "config.json").read_text()
    )
    for k, v in config.items():
        flat_config[f"model.config.{k}"] = v

    for k, v in list(flat_config.items()):
        if isinstance(v, (list, tuple)):
            try:
                flat_config[k] = ".".join(_v for _v in v if _v)
            except:
                del flat_config[k]
        elif isinstance(v, dict):
            del flat_config[k]

    return flat_config


def load_run(run_file):
    run = pd.read_csv(
        run_file,
        sep=r"\s+",
        header=None,
        names=["query_id", "Q0", "doc_id", "rank", "score", "run_name"],
    )
    run = run.sort_values(["query_id", "score"], ascending=[True, False])
    run["rank"] = run.groupby("query_id")["score"].rank(ascending=False, method="first")
    
    if run_file.parent.parent.name == "tirex" or "rank-" in run_file.parent.name:
        run["run_name"] = run_file.parent.name

    config = {}
    config_path = run_file.parent.parent.parent / "pl_config.yaml"
    if config_path.exists() and "wandb" in str(run_file):
        run_id = run_file.parent.parent.parent.parent.name.split("-")[-1]
        config = parse_config(config_path)
        run["run_name"] = run_id
        
    config["run_name"] = run.iloc[0]["run_name"]
    dataset_name = run_file.name.split(".")[0].strip("_")
    dataset_name = re.sub(r"_+", "_", dataset_name)
    try:
        dataset_id = DASHED_DATASET_MAP[dataset_name]
        first_stage = "tirex"
        if run_file.parent.name in ("bm25", "chatnoir"):
            config["run_name"] = "first_stage"
        if run_file.parent.name == "colbert":
            config["run_name"] = "first_stage"
            first_stage = "colbert"
    except KeyError:
        split = dataset_name.split("_")
        if len(split) == 2:
            first_stage, dataset_name = split
        else:
            raise ValueError(f"Unknown dataset name: {dataset_name}")
        if "tirex" in first_stage:
            first_stage = "tirex"
        dataset_id = DASHED_DATASET_MAP[dataset_name]

    config["dataset"] = dataset_id
    config["first_stage"] = first_stage
    config["base"] = docs_parent_id(dataset_id)
    run = run.astype({"query_id": str, "doc_id": str})
    return run, config


@lru_cache
def load_qrels(dataset):
    dataset = ir_datasets.load(dataset)
    qrels_df = pd.DataFrame(dataset.qrels_iter())
    return qrels_df


In [23]:
metrics = {
    "nDCG@10": {},
    # "NDCG@10_UNJ" : {"removeUnjudged": True},
}

In [31]:
# trectools throws annoying warnings because of pandas
warnings.simplefilter(action="ignore", category=FutureWarning)
values = []

run_files = (
    list(BASELINE_DIR.glob("tirex/*/*.run"))
    + list(BASELINE_DIR.glob("colbert/*.run"))
    + list(BASELINE_DIR.glob("rank-*/*.run"))
    + list(LOG_DIR.glob("**/*.run"))
)

pg = tqdm(run_files)
for run_file in pg:
    if (
        "train" in run_file.name
        or "msmarco-passage-v2" in run_file.name
        or "beir" in run_file.name
        or "dev" in str(run_file)
        or "tripclick" in run_file.name
        or run_file.name.startswith("__")
        or "msmarco-document" in run_file.name
        # or "tirex" in str(run_file)
        # or "orcas" in run_file.name
        # or ("dev" in run_file.name and "small" not in run_file.name)
    ):
        continue
    if "diversity" in run_file.name:
        continue
    pg.set_description(
        str(
            run_file.relative_to(BASELINE_DIR)
            if run_file.is_relative_to(BASELINE_DIR)
            else run_file.relative_to(LOG_DIR)
        )
    )
    run_df, config = load_run(run_file)
    if run_df is None:
        continue
    qrels_df = load_qrels(config["dataset"])
    qrel_qids = set(qrels_df["query_id"])
    run_qids = set(run_df["query_id"].drop_duplicates())
    missing_qids = set(qrel_qids) - set(run_qids)
    if missing_qids:
        # print(f"found {len(missing_qids)} missing qids in {run_file}")
        qrels_df = qrels_df.loc[qrels_df["query_id"].isin(run_qids)]
    _metrics = evaluate_run(run_df, qrels_df, metrics)
    values.append({**config, **_metrics})

results = pd.DataFrame(values)
# per_query_results = pd.concat(per_query_dfs).infer_objects().reset_index(drop=True)

# per_query_results = per_query_results.loc[
#     ~(
#         per_query_results["first_stage"].str.contains("tirex")
#         & per_query_results["dataset"].str.contains("msmarco")
#     )
# ]

# del per_query_dfs

# per_query_results["finetuned"] = per_query_results[
#     "model.model_name_or_path"
# ].str.contains("experiments")

# per_query_results.to_json("per_query_results.jsonl", orient="records", lines=True)

tirex/monot5-large/clueweb12-touche-2020-task-2.run:   1%|          | 8/1055 [00:08<12:35,  1.39it/s]       

/var/tmp/fschlatt/.ir_datasets/touche/2020/task-2/qrels.qrels


tirex/monot5-large/argsme-2020-04-01-touche-2020-task-1.run:   2%|▏         | 16/1055 [00:14<15:42,  1.10it/s]

/var/tmp/fschlatt/.ir_datasets/touche/2020/task-1/qrels.qrels


run-20240424_120056-008cf7pt/files/huggingface_checkpoint/runs/tirex-rerank_gov2-trec-tb-2005.run: 100%|██████████| 1055/1055 [05:11<00:00,  3.39it/s]                          


In [16]:
def run_type(row):
    if not pd.isna(row["model.model_name_or_path"]):
        return "monoELECTRA"
    if "rank-gpt" in row["run_name"]:
        if "turbo" in row["run_name"]:
            return "RankGPT-4 Turbo"
        return "RankGPT-4"
    if "rank-zephyr" in row["run_name"]:
        return "RankZephyr"
    if "monot5" in row["run_name"]:
        return row["run_name"].replace("t5", "T5").replace("-", " ")
    if "monobert" in row["run_name"]:
        return row["run_name"].replace("bert", "BERT").replace("-", " ")
    if "sparse" in row["run_name"]:
        return "Sparse monoMiniLM"
    if "list-in-t5" in row["run_name"]:
        return "LiT5-Distill"
    return "First Stage"


def num_params(row):
    if "monot5-base" in row["run_name"]:
        return "220M"
    if "monot5-large" in row["run_name"]:
        return "770M"
    if "monot5-3b" in row["run_name"]:
        return "3B"
    if "rank-gpt-4" in row["run_name"]:
        return "?"
    if "rank-zephyr" in row["run_name"]:
        return "7B"
    if "list-in-t5" in row["run_name"]:
        return "220M"
    if "sparse" in row["run_name"]:
        return "11M"
    if "monobert-base" in row["run_name"]:
        return "110M"
    if "monobert-large" in row["run_name"]:
        return "340M"
    if (
        not pd.isna(row["model.model_name_or_path"])
        and row["model.config.num_hidden_layers"] == 12
    ):
        return "110M"
    if (
        not pd.isna(row["model.model_name_or_path"])
        and row["model.config.num_hidden_layers"] == 24
    ):
        return "340M"
    return "--"


def first_train_dataset(row):
    if pd.isna(row["data.train_dataset"]):
        return None
    if "google/electra" not in row["model.model_name_or_path"]:
        return "CBv2"
    if "rank-gpt-3-turbo" in row["data.train_dataset"]:
        return "RGPT3.5"
    if "rank-gpt-4-turbo" in row["data.train_dataset"]:
        return "RGPT4-T"
    if "msmarco-passage-train" in row["data.train_dataset"]:
        return "CBv2"
    if "twolar" in row["data.train_dataset"]:
        return "TWOLAR"
    raise ValueError(f"Unknown dataset: {row['data.train_dataset']}")


def second_train_dataset(row):
    if pd.isna(row["data.train_dataset"]):
        return None
    if row["first_train_dataset"] != "CBv2":
        return None
    if "rank-gpt-3-turbo" in row["data.train_dataset"]:
        return "RGPT3.5"
    if "rank-gpt-4-turbo" in row["data.train_dataset"]:
        return "RGPT4-T"
    if "msmarco-passage-train" in row["data.train_dataset"]:
        return "CBv2"
    if "twolar" in row["data.train_dataset"]:
        return "TWOLAR"
    raise ValueError(f"Unknown dataset: {row['data.train_dataset']}")


results["run_type"] = results.apply(run_type, axis=1)
results["first_train_dataset"] = results.apply(first_train_dataset, axis=1)
results["second_train_dataset"] = results.apply(second_train_dataset, axis=1)
results["num_params"] = results.apply(num_params, axis=1)

In [17]:
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
groupby_columns = list(
    filter(
        lambda x: x not in list(metrics) + ["dataset"],
        results.columns,
    )
)
columns = results.columns.intersection(list(metrics)).tolist()
base_results = (
    results.groupby(groupby_columns, dropna=False)[columns]
    .mean()
    .reset_index()
    .copy()
)
base_results

,run_name,first_stage,base,seed_everything,trainer.accelerator,trainer.strategy,trainer.devices,trainer.num_nodes,trainer.precision,trainer.logger.class_path,...,model.config.architectures,model.config.model_type,model.config.save_step,model.config.torch_dtype,model.config.transformers_version,run_type,first_train_dataset,second_train_dataset,num_params,nDCG@10
0,008cf7pt,colbert,msmarco-passage,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,590.0,float32,4.39.3,monoELECTRA,CBv2,TWOLAR,110M,0.761399
1,008cf7pt,tirex,antique,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,590.0,float32,4.39.3,monoELECTRA,CBv2,TWOLAR,110M,0.576209
2,008cf7pt,tirex,argsme/2020-04-01,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,590.0,float32,4.39.3,monoELECTRA,CBv2,TWOLAR,110M,0.305200
3,008cf7pt,tirex,clueweb09/en,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,590.0,float32,4.39.3,monoELECTRA,CBv2,TWOLAR,110M,0.185766
4,008cf7pt,tirex,clueweb12,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,590.0,float32,4.39.3,monoELECTRA,CBv2,TWOLAR,110M,0.292334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,zhub2hz2,tirex,medline/2017,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,189.0,float32,4.39.3,monoELECTRA,CBv2,RGPT4-T,110M,0.309761
482,zhub2hz2,tirex,msmarco-passage,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,189.0,float32,4.39.3,monoELECTRA,CBv2,RGPT4-T,110M,0.719828
483,zhub2hz2,tirex,nfcorpus,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,189.0,float32,4.39.3,monoELECTRA,CBv2,RGPT4-T,110M,0.298179
484,zhub2hz2,tirex,vaswani,42.0,auto,auto,auto,1.0,bf16-mixed,lightning_ir.main.CustomWandbLogger,...,FlashMonoElectraModel,mono-electra,189.0,float32,4.39.3,monoELECTRA,CBv2,RGPT4-T,110M,0.525003


In [21]:
results.filter(like="loss", axis=1)

""
0
1
2
3
4
...
997
998
999
1000


In [8]:
index_columns = [
    "data.train_dataset",
    "trainer.max_epochs",
    "run_name",
]
columns = ["first_stage", "dataset"]
values = ["nDCG@10"]

filter_series = (
    results["dataset"].str.contains("msmarco") &
    results["first_stage"].isin(["tirex", "colbert"])
    & results["model.config.loss_functions"]
)

table = (
    results.loc[filter_series]
    .pivot(
        index=index_columns,
        columns=columns,
        values=values,
    )
    .droplevel(axis=1, level=0)
)

table.sort_index(ascending=False).dropna().round(3)

KeyError: 'model.config.loss_functions'

In [ ]:
index_columns = [
    "data.train_dataset",
    "trainer.max_epochs",
    "run_name",
]
columns = ["first_stage", "base"]
values = ["nDCG@10"]

filter_series = (
    base_results["first_stage"].eq("tirex")
)

table = (
    base_results.loc[filter_series]
    .pivot(
        index=index_columns,
        columns=columns,
        values=values,
    )
    .droplevel(axis=1, level=[0, 1])
)

def hmean(x):
    return 1 / np.mean(1 / x)


def qmean(x):
    return np.sqrt(np.mean(x**2))


def gmean(x):
    return np.exp(np.log(x.prod()) / x.notna().sum(1))


arithmetic_mean = table.mean(axis=1).rename("Arithmetic Mean").to_frame()
# harmonic_mean = table.apply(hmean, axis=1).rename("Harmonic Mean").to_frame()
# quadratic_mean = table.apply(qmean, axis=1).rename("Quadratic Mean").to_frame()
geoemtric_mean = table.apply(qmean, axis=1).rename("Geometric Mean").to_frame()

table = pd.concat(
    [
        table,
        arithmetic_mean,
        # harmonic_mean,
        # quadratic_mean,
        geoemtric_mean,
    ],
    axis=1,
)


table.sort_index(ascending=False).dropna().round(3).sort_values("Geometric Mean", ascending=False)

antique  \
data.train_dataset                                 trainer.max_epochs run_name                               
NaN                                                NaN                rank-zephyr                    0.534   
data/baseline-runs/rank-zephyr/__colbert-10000-... 3.0                q7d4u8sj                       0.581   
                                                                      lb5en990                       0.584   
                                                                      8ntakfv1                       0.583   
data/baseline-runs/rank-gpt-3-turbo/msmarco-pas... 1.0                0seh1a6k                       0.470   
data/baseline-runs/rank-zephyr/__colbert-10000-... 1.0                s7n25zed                       0.588   
NaN                                                NaN                monot5-3b                      0.543   
                                                                      castorini-list-in-t5-300       0.576   
data/baseline-runs/rank-zephyr/__colbert-10000-... 1.0                kw5kywed                       0.573   
data/baseline-runs/colbert/msmarco-passage-trai... NaN                monoelectralarge               0.492   
data/baseline-runs/rank-gpt-4-turbo/__colbert-2... 3.0                zhub2hz2                       0.570   
                                                                      wx2s6bqk                       0.570   
data/baseline-runs/twolar/twolar.jsonl             1.0                008cf7pt                       0.576   
data/baseline-runs/rank-gpt-4-turbo/__colbert-2... 3.0                jpy50xym                       0.570   
NaN                                                NaN                monot5-large                   0.532   
data/baseline-runs/twolar/twolar.jsonl             1.0                c72xi2de                       0.570   
data/baseline-runs/rank-gpt-4-turbo/__colbert-2... 1.0                awp7b89h                       0.578   
NaN                                                NaN                sparse-cross-encoder-4-512     0.545   
data/baseline-runs/rank-gpt-3-turbo/msmarco-pas... 1.0                55expd37                       0.350   
NaN                                                NaN                monobert-large                 0.489   
                                                                      monobert-base                  0.512   
                                                                      monot5-base                    0.510   
data/baseline-runs/colbert/msmarco-passage-trai... NaN                monoelectrabase                0.517   
NaN                                                NaN                first_stage                    0.516   
data/baseline-runs/colbert/msmarco-passage-trai... NaN                monoelectrasmall               0.448   
NaN                                                NaN                sparse-cross-encoder-4-4096    0.175   
data/baseline-runs/rank-gpt-4-turbo/__colbert-2... 1.0                06me6940                       0.321   

                                                                                                   argsme/2020-04-01  \
data.train_dataset                                 trainer.max_epochs run_name                                         
NaN                                                NaN                rank-zephyr                              0.363   
data/baseline-runs/rank-zephyr/__colbert-10000-... 3.0                q7d4u8sj                                 0.335   
                                                                      lb5en990                                 0.333   
                                                                      8ntakfv1                                 0.328   
data/baseline-runs/rank-gpt-3-turbo/msmarco-pas... 1.0                0seh1a6k                                 0.420   
data/baseline-runs/rank-zephyr/__colbert-10000-... 1.0              